In [1]:
import numpy as np
import heapq

In [10]:
# Generates the final state (puzzle solved)
def end_state(n):

    # Create a list of numbers from 1 to n^2 - 1
    elements = np.arange(1, n**2)

    # Append a zero at the end
    elements = np.append(elements, 0)

    # Reshape into an n x n matrix
    end_state_matrix = elements.reshape(n, n)

    return end_state_matrix


# Finds the position of the zero element in the matrix and determines all possible moves for it
def available_actions(state, n):
    
    moves = []  # List to store possible moves

    # Find the position of the zero element
    zero_pos = tuple(np.argwhere(state == 0)[0])

    # Possible move directions: (row_change, col_change)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for dr, dc in directions:
        new_row, new_col = zero_pos[0] + dr, zero_pos[1] + dc

        # Check if the move is within bounds
        if 0 <= new_row < n and 0 <= new_col < n:
            moves.append((new_row, new_col))

    # Return the current position of zero and its possible moves
    return [zero_pos] + moves


# Generates random solvable starting state
def generate_random_estate(state, n):

    random_moves = n * 100  

    random_estate = state.copy()

    moves = 0

    while moves <= random_moves:
        moves += 1

        # Get available actions
        possible_actions = available_actions(random_estate, n)

        # Choose a random action
        random_action = possible_actions[np.random.randint(1, len(possible_actions))]

        # Move zero
        zero_i, zero_j = possible_actions[0]
        new_zero_i, new_zero_j = random_action
        random_estate[zero_i, zero_j], random_estate[new_zero_i, new_zero_j] = random_estate[new_zero_i, new_zero_j], random_estate[zero_i, zero_j]


    return random_estate

In [11]:
# Manhattan distance with linear conflict
def manhattan_distance_with_linear_conflict(state, goal_state):
    distance = 0
    linear_conflict = 0

    for i in range(state.shape[0]):
        for j in range(state.shape[1]):
            if state[i, j] != 0:
                # Manhattan Distance
                goal_pos = np.argwhere(goal_state == state[i, j])[0]
                distance += abs(i - goal_pos[0]) + abs(j - goal_pos[1])

                # Check for Linear Conflict in the row
                if i == goal_pos[0]:  # Same row
                    for k in range(j + 1, state.shape[1]):
                        if state[i, k] != 0:
                            goal_k_pos = np.argwhere(goal_state == state[i, k])[0]
                            if goal_k_pos[0] == i and goal_k_pos[1] < goal_pos[1]:
                                linear_conflict += 1

                # Check for Linear Conflict in the column
                if j == goal_pos[1]:  # Same column
                    for k in range(i + 1, state.shape[0]):
                        if state[k, j] != 0:
                            goal_k_pos = np.argwhere(goal_state == state[k, j])[0]
                            if goal_k_pos[1] == j and goal_k_pos[0] < goal_pos[0]:
                                linear_conflict += 1

    # Each linear conflict requires 2 extra moves
    return distance + 2 * linear_conflict



# A* algorithm
def a_star(init_state, goal_state, n):
    
    # Priority queue for states, initialized with the start state
    pq = []
    heapq.heappush(pq, (0, tuple(init_state.flatten()), 0, None))  # (f(n), state as tuple, g(n), previous_state)

    # Track visited states and their predecessors
    visited = {}  # Maps a state tuple to its predecessor
    visited[tuple(init_state.flatten())] = None

    # Cost and quality counters
    cost = 0  # Number of states evaluated (added to the queue)
    quality = 0  # Number of actions in the solution path

    while pq:
        # Pop the state with the lowest f(n)
        f_n, current_state_tuple, g_n, prev_state_tuple = heapq.heappop(pq)

        # Convert current state back to matrix
        current_state = np.array(current_state_tuple).reshape(init_state.shape)

        # Check if the goal state is reached
        if np.array_equal(current_state, goal_state):
            quality = g_n  # The number of steps taken to reach the goal

            # Backtrack to reconstruct the path
            path = []
            state_tuple = current_state_tuple
            while state_tuple is not None:
                path.append(state_tuple)
                state_tuple = visited[state_tuple]

            # Print the results
            print("Goal reached!")
            print(f"Cost (states evaluated): {cost}")
            print(f"Quality (solution path length): {quality}")
            print("\nSolution Path (from initial to goal):")
            for step, state in enumerate(reversed(path)):
                print(f"Step {step}:")
                print(np.array(state).reshape(init_state.shape))
            return

        # Find all possible moves for the zero tile
        possible_actions = available_actions(current_state, n)

        for action in possible_actions[1:]:
            # Create the new state by moving zero
            new_state = current_state.copy()
            zero_i, zero_j = possible_actions[0]
            new_i, new_j = action
            new_state[zero_i, zero_j], new_state[new_i, new_j] = new_state[new_i, new_j], new_state[zero_i, zero_j]

            # Convert to tuple for hashable state
            new_state_tuple = tuple(new_state.flatten())

            # Ensure that only unvisited states are pushed
            if new_state_tuple not in visited:
                visited[new_state_tuple] = current_state_tuple  # Record the predecessor
                h_n = manhattan_distance_with_linear_conflict(new_state, goal_state)
                f_n = g_n + 1 + h_n
                heapq.heappush(pq, (f_n, new_state_tuple, g_n + 1, current_state_tuple))
                cost += 1  # Increment cost when a new state is added to the queue


In [12]:
PUZZLE_DIM_N = 4

In [13]:
# Test the A* algorithm
goal_state = end_state(PUZZLE_DIM_N)
init_state = generate_random_estate(goal_state, PUZZLE_DIM_N)

print("Initial State:")
print(init_state)
print("\nGoal State:")
print(goal_state, '\n')

a_star(init_state, goal_state, PUZZLE_DIM_N)

Initial State:
[[10  6 11  7]
 [ 9  8  0 13]
 [ 2 15  3 12]
 [14  5  1  4]]

Goal State:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]] 

Goal reached!
Cost (states evaluated): 3889986
Quality (solution path length): 53

Solution Path (from initial to goal):
Step 0:
[[10  6 11  7]
 [ 9  8  0 13]
 [ 2 15  3 12]
 [14  5  1  4]]
Step 1:
[[10  6 11  7]
 [ 9  8  3 13]
 [ 2 15  0 12]
 [14  5  1  4]]
Step 2:
[[10  6 11  7]
 [ 9  8  3 13]
 [ 2 15  1 12]
 [14  5  0  4]]
Step 3:
[[10  6 11  7]
 [ 9  8  3 13]
 [ 2 15  1 12]
 [14  5  4  0]]
Step 4:
[[10  6 11  7]
 [ 9  8  3 13]
 [ 2 15  1  0]
 [14  5  4 12]]
Step 5:
[[10  6 11  7]
 [ 9  8  3  0]
 [ 2 15  1 13]
 [14  5  4 12]]
Step 6:
[[10  6 11  0]
 [ 9  8  3  7]
 [ 2 15  1 13]
 [14  5  4 12]]
Step 7:
[[10  6  0 11]
 [ 9  8  3  7]
 [ 2 15  1 13]
 [14  5  4 12]]
Step 8:
[[10  6  3 11]
 [ 9  8  0  7]
 [ 2 15  1 13]
 [14  5  4 12]]
Step 9:
[[10  6  3 11]
 [ 9  8  1  7]
 [ 2 15  0 13]
 [14  5  4 12]]
Step 10:
[[10  6  3 11]
 [ 9  8  1  7